In [1]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir



import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

from sklearn import linear_model
from scipy import stats
import numpy as np
import matplotlib.patches as mpatches


# visualize results module
from visualize import visualize_svm as Visualize

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


Autosaving every 180 seconds


In [2]:
##########################################################################
##########################################################################
##########################################################################

def compute_significance(data,
                         significance):

        #print ("self.data: ", data.shape)
        
        mean = data.mean(1)
        #
        sig = []
        for k in range(data.shape[0]):
            #res = stats.ks_2samp(self.data[k],
            #                     control)
            #res = stats.ttest_ind(first, second, axis=0, equal_var=True)

            #
            res = scipy.stats.ttest_1samp(data[k], 0.5)

            sig.append(res[1])


        sig_save = np.array(sig).copy()
        #print ("Self sig save: ", sig_save.shape)

        # multiple hypothesis test
        temp = np.array(sig)
        #print ("data into multi-hypothesis tes:", temp.shape)
        temp2 = multipletests(temp, alpha=significance, method='fdr_bh')
        sig = temp2[1]

        #
        sig=np.array(sig)[None]

        #
        thresh = significance
        idx = np.where(sig>thresh)
        sig[idx] = np.nan

        #
        idx = np.where(mean<0.5)
        sig[:,idx] = np.nan
        #print ("SIG: ", sig.shape)

        # find earliest 
        earliest_continuous = 0
        for k in range(sig.shape[1]-1,0,-1):
            if sig[0][k]<=significance:
                earliest_continuous = k
            else:
                break
                
        earliest_continuous = -(sig.shape[1]-earliest_continuous)/30.
                
        return sig, earliest_continuous
    
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self
    
 

def plot_edt_distributions():

    fig = plt.figure(figsize=(20,12))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(2,3,ctr+1)
        fnames = []
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_lockout.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')

        # 
        shifts=[+15,+15,0]
        linestyles = ['-','-','-']
        titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []
        for ctr_type, fname in enumerate(fnames):
            #print (" ", animal_id, fname)
            data = np.load(fname, allow_pickle=True)

            # 
            clr = vis.colors[ctr_type]
            res = vis.plot_first_decoding_time_curves(data,
                                                        titles[ctr_type],
                                                        shifts[ctr_type],
                                                        linestyles[ctr_type],
                                                        clr)
            edts.append(res[0])
            yvals.append(res[1])

        if ctr==0:
            plt.legend(fontsize=legend_size)

        temp = np.hstack(yvals)
        ymax = np.max(temp)
        #print ("ymax", ymax)
        #
        plt.xlim(-15,0)
        plt.ylim(bottom=0)    
        plt.xticks([])
        plt.yticks([])
        plt.plot([-3,-3],[0,ymax*1.1],'--',linewidth=5,c='grey')
        plt.plot([-10,-10],[0,ymax*1.1],'--',linewidth=5,c='grey')
        #
        #ax2=
        ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

        patches = []
        res = stats.ks_2samp(edts[0], edts[1])
        print (res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='blue', label=label_))

        res = stats.ks_2samp(edts[0], edts[2])
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='red', label=label_))

        #print ('')
        plt.legend(handles=patches,
                   fontsize=legend_size)



        plt.xlim(-15,0)
        plt.ylim(bottom=0)
        plt.xticks([])
        plt.yticks([])

    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first.svg',dpi=1200)
        plt.close()
        
     

In [9]:
##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize.Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.code = 'left_paw'
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True


In [56]:
##########################################################
######## PLOT SINGLE SESSION SVM DECODING SESSION ########
##########################################################
vis.significance = 0.05
vis.linewidth=10

#
vis.cbar_offset = 0
vis.smooth_window = 30
vis.lockout = False
vis.xvalidation = 10
vis.sliding_window = 30

# 
vis.min_trials = 10

############################
# Fig 2E
vis.window = 30

# examples > 10 sec
#vis.animal_id = "AQ2"
#vis.session_id = 'Feb12_'
#vis.session_id = 'Mar15_'
#vis.session_id = 'Mar18_'
#vis.session_id = 'Apr4_'
#vis.session_id = 'Apr22_'


vis.cbar_thick = 0.05

# # 


# # 
# vis.save_fname = '/home/cat/predict.png'

alpha = 1.0
#sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]
session = 'IJ2pm_Mar3_30Hz'
vis.animal_id = 'IJ2'
vis.fname = None

vis.code = 'code_04'
vis.pickle = False
plot_single = False

vis.fig=plt.figure(figsize=(10,10))
vis.ax = plt.subplot(111)

if plot_single:
    print ("SESSION: ", session)
    fname_out = 'single'
    vis.n_trials_plotting=[]
    vis.shift_SVM = True
    vis.n_trials=200
    vis.session_id = session
    vis.shift_flag = False
    vis.apply_lever_offset = False   # this is for sihfting DLC triggered data
    vis.xlim = [-15,0]
    vis.show_EDT = True
    vis.edt_offset_y = .1
    vis.edt_offset_x = 1.5#
    vis.show_title = False
    vis.show_legend = False
    vis.legend_location = 1
    vis.cbar=False

    #vis.get_sessions()
    #print (vis.sessions)
    # 
    vis.cmap = "Blues_r"
    vis.alpha = alpha
    vis.plot_significant('blue',vis.session_id)

else:
    fname_out = 'concatenated'

#     print ("")
#     vis.session_id = 'concatenated'
# #     vis.fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenatedcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30_5PCA_5PCA.npz'
# #     vis.cmap = "Greens_r"
#     vis.title = '5pca'
#     vis.title_offset = 0
#     vis.cbar=True
# #     vis.plot_significant_concatenated('green',vis.session_id)

    vis.n_trials=200
    vis.session=session
    vis.session='concatenated'
    vis.session_id = 'concatenated'
    vis.fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenated_code_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
    vis.cmap = "Reds_r"
    vis.xlim = [-15,0]
    vis.title = '20pca'
    vis.edt_offset_y = .1
    vis.edt_offset_x = 3
    vis.cbar = False
    vis.title_offset = -0.05
    vis.plot_significant_concatenated('red',vis.session_id)

    vis.cbar_offset=0

if True:
    plt.savefig('/home/cat/'+session+'.svg',dpi=1200)
    plt.close()
else:
    plt.show()


loaded data:  (870, 10)
self.data:  (841, 10)
Self sig save:  (841,)
data into multi-hypothesis tes: (841,)
Final sig:  (1, 841)
earliest:  459  in sec:  -12.733333333333333
 signianc n-trials:  200
self.sig.shape (1, 841)
earliest:  69  in sec:  -12.7


In [143]:
###############################################################
####### GENERATE LONGITUDINAL SVM CURVES (VIRIDIS PLOTS) ######
###############################################################

# 6 panels with SVM curves in vidiris; ALREADY GEENERATED
names = ['IA1','IA2','IA3','IJ1','IJ2', 'AQ2']


fig, ax = plt.subplots()
import scipy
from statsmodels.stats.multitest import multipletests

smooth = True
smooth_window = 30
significance = 0.05
auc = []
early = []
from tqdm import tqdm, trange
for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    auc.append([])
    early.append([])
    for ctr,fname in tqdm(enumerate(fnames)):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)
            data = np.array(data)
            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        #print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())
        
        sig, earliest = compute_significance(data.T,
                                             significance)
        early[k].append(earliest)
        
    plt.ylim(0.4,1.0)
    plt.xlim(-15,0)
    plt.plot([-30,0],[0.5,0.5],'--',c='black')

if True:
    plt.savefig('/home/cat/edt_trends.svg',dpi=1200)
    plt.close()


46it [00:13,  3.30it/s]
33it [00:09,  3.31it/s]
41it [00:12,  3.33it/s]
40it [00:11,  3.35it/s]
39it [00:11,  3.35it/s]
78it [00:23,  3.36it/s]


In [168]:
#########################################
########### EDT FITS ####################
#########################################


# EDT FITS

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig=plt.figure(figsize=(6,6))
for k in range(len(early)):
#for k in [3]:
    ax=plt.subplot(3,2,k+1)
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])/temp.shape[0]
    
    #
    lr = LinearRegression()
    lr.fit(t.reshape(-1, 1), temp.reshape(-1, 1))
    #print (k, temp)
    #print (k,"tsta: ", lr.t, "pval: ", lr.p)


    
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             #label=ids[k]+ " ***** ",
             c='black')

    # 
    corr = scipy.stats.pearsonr(t,temp)
    print ("corrL: ", corr)
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                #linewidth=4,
                c='grey',
                alpha=.8,
               label = str(round(corr[0],2)))
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)

    plt.xticks([])
    plt.yticks([])
    legend = plt.legend(handlelength=0, fontsize=12)
    
    #plt.legend(fontsize=12)
    plt.ylim(-12,0)

#
    plt.xlim(0,1)

if True:
    plt.savefig('/home/cat/edt_fits.svg', dpi=600)
    plt.close()
else:
    plt.show()


corrL:  (0.7014331655212428, 5.6406315699993825e-08)
corrL:  (-0.542035486329535, 0.0011210149237215162)
corrL:  (-0.3419947074557963, 0.02862744552301875)
corrL:  (0.020071514436453608, 0.9021622977083262)
corrL:  (-0.1292660053033484, 0.43286401776660616)
corrL:  (-0.3602431759059126, 0.0011964553177763325)


In [12]:
#############################################################
### COMPUTE DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ##
#############################################################
from tqdm import trange

##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize.Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.code = 'left_paw'
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True

#################################################
# GENERATE DATA FOR ACCORDEON PLOTS or EDT CURVES
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 30          # # sec out e.g. -30sec ..+30sec
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False
vis.imaging_rate = 30.
vis.pca_var = 0.95
vis.pca_flag = True
vis.xvalidation = 10
vis.sliding_window = 30   # # of frames to put into SVM data e.g. -1sec .. 0 == 30 frames

# 
vis.smooth_window = 30   # this is a window for applying filter to output
vis.min_trials = 20
vis.xlim=[-15,0]
vis.pickle = False
vis.shift_flag = False
vis.apply_lever_offset = False
vis.dlc_offset_flag = False
#
vis.significance=0.01

#
vis.compute_roi = False

#
vis.regular_flag = True
vis.concatenated_flag = False
vis.ideal_window_flag = False

# 
lockouts = [True]

vis.animal_ids = ['IA1','IA2','IA3','IJ1','IJ2', 'AQ2']


# compute regular sessions
if vis.regular_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time(lockouts)

# compute concatenated sessions
elif vis.concatenated_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time_concatenated(lockouts)

# 
elif vis.ideal_window_flag:
    for animal_id in vis.animal_ids:
        vis.animal_id = animal_id
        vis.compute_first_decoding_time(lockouts)
    

    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb1_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb4_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb5_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb10_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb12_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb18_30Hzcode_04_lockout_10sec_trial_ROItime

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb18_30Hz/IA1pm_Feb18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb18_30Hz/IA1pm_Feb18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb19_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  73
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Apr18_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  14
Insufficient trials... 14
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Apr19_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  20
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Apr20_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  16
Insufficient trials... 16
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Sco

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA2/SVM_Scores/SVM_Scores_IA2pm_Feb5_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  69
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb9_30Hz/IA2pm_Feb9_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb9_30Hz/IA2pm_Feb9_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb10_30Hz/IA2pm_Feb10_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb10_30Hz/IA2pm_Feb10_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz


 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA2/SVM_Scores/SVM_Scores_IA2pm_Mar17_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  85
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA2/SVM_Scores/SVM_Scores_IA2pm_Mar18_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  64
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IA2/SVM_Scores/SVM_Scores_IA2pm_Mar21_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  51
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:

...

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar1_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  67
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  110
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_IJ2pm_Mar3_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  52
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slide

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2pm_Jan28_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  61
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Jan29_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  41
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2pm_Jan29_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  51
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2pm_Mar15_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  145
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Mar16_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  123
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Mar17_Week2_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  105
(1742,)
self mean:  (450,)
 ... finished se

 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Apr25_Week8_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  114
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Apr26_Week8_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  112
(1742,)
self mean:  (450,)
 ... finished session...

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_AQ2am_Apr27_Week8_30Hzcode_04_lockout_10sec_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  138
(1742,)
self mean:  (450,)
 ... finished se

In [14]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# REPLACEMENT OF ACORDEON PLOTS

#plot_edt_distributions()        
import pylab as P


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_edt_distributions_box_plots(vis):

    vis = Visualize.Visualize()
    vis.regular_flag = True
    vis.ideal_window_flag = False
    #vis.animal_ids = ['IJ1']
    
    fig = plt.figure(figsize=(20,12))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(2,3,ctr+1)
        fnames = []

        # plots for Fig 2 comparing 
        if vis.regular_flag:
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_lockout.npz')
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')
        elif vis.ideal_window_flag:
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_ideal_window.npz')
            
        # 
        shifts=[+15,+15,0]
        linestyles = ['-','-','-']
        titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []
        plotting = False

        for ctr_type, fname in enumerate(fnames):
            #print (" ", animal_id, fname)
            data = np.load(fname, allow_pickle=True)

            # 
            clr = vis.colors[ctr_type]
            res = vis.plot_first_decoding_time_curves(data,
                                                      titles[ctr_type],
                                                      shifts[ctr_type],
                                                      linestyles[ctr_type],
                                                      clr, 
                                                      plotting)
            

            temp = res[0]
            # remove low values; don't use fo rnow
            if False:
                idx = np.where(temp<-0.5)[0]
                temp=temp[idx]

            #    
            edts.append(temp)
            yvals.append(res[1])

        
        my_dict = dict(all = edts[0], 
                       lockout = edts[1],
                       concat = edts[2]
                      )
        
        data = pd.DataFrame.from_dict(my_dict, orient='index')
        data = data.transpose()


        flierprops = dict(marker='o', 
                          #markerfacecolor='g', 
                          markersize=10000,
                          linestyle='none', 
                          markeredgecolor='r')
        
        #
        data.boxplot(#showfliers=False,
                    flierprops=flierprops)
        clrs_local = ["blue","black","red"]
        for i,d in enumerate(data):
            y = data[d]
            x = np.random.normal(i+1, 0.04, len(y))
            if True:
                plt.plot(x, y, 
                     mfc =clrs_local[i], 
                     mec='k', 
                     ms=7, 
                     marker="o", 
                     linestyle="None",
                     
                    )
            else:
                plt.scatter(x, y, 
                       c=clrs_local[i],
                       edgecolor='black',
                       s=200,
                       alpha=.7)

            
        #if ctr==0:
        #    plt.legend(fontsize=legend_size)

        temp = np.hstack(yvals)
        ymax = np.max(temp)

        #
        plt.xlim(0.5, 3.5)
        plt.ylim(-15,0)    
        plt.xticks([])
        plt.yticks([])

        #
        if True:
        
            plt.plot([0,3.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
            plt.plot([0,3.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)

            #ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

            patches = []
            res = stats.ks_2samp(edts[0], edts[1])
            print (res)
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='blue', label=label_))

            res = stats.ks_2samp(edts[0], edts[2])
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='red', label=label_))

            #print ('')
            
            plt.legend(handles=patches,
                       fontsize=legend_size)


    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first_scatter.svg')
        plt.close()
        
        
plot_edt_distributions_box_plots(vis)


temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (53,)
KstestResult(statistic=0.41935483870967744, pvalue=0.007945718302533482)
temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (31,)
temp data allrescontinuous:  (33,)
KstestResult(statistic=0.1935483870967742, pvalue=0.615053935325554)
temp data allrescontinuous:  (38,)
temp data allrescontinuous:  (38,)
temp data allrescontinuous:  (41,)
KstestResult(statistic=0.2631578947368421, pvalue=0.14451472018143485)
temp data allrescontinuous:  (33,)
temp data allrescontinuous:  (33,)
temp data allrescontinuous:  (40,)
KstestResult(statistic=0.30303030303030304, pvalue=0.09654611183078916)
temp data allrescontinuous:  (36,)
temp data allrescontinuous:  (36,)
temp data allrescontinuous:  (39,)
KstestResult(statistic=0.1388888888888889, pvalue=0.8848803300554184)
temp data allrescontinuous:  (76,)
temp data allrescontinuous:  (76,)
temp data allrescontinuous:  (78,)
KstestResult(st

In [126]:
############################################################
#### PLOT CORRELATION OF DECODING TIME AND NO. SESSIONS ####
############################################################
from tqdm import trange
import matplotlib.cm as cm

vis.smooth_window = 30
vis.min_trials = 20
vis.animal_id = 'IA1'
vis.session = "Feb1_"
vis.window = 30
vis.xlim=-15

#
fig = plt.figure(figsize=(10,8))
vis.significance=0.05 #1E-2
vis.lockout=False
vis.ylims=[200,400]
vis.cmap=cm.Reds
vis.plot_legend = False
# 
clr='blue'

# TWO OPTIONS: singles or contatnated
#fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_all_30sec_backup.npz'
fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_lockout_30sec_concatenated_backup.npz'
vis.plot_first_decoding_time_vs_n_trials(clr, fname)

# #
if False:
    plt.show()
else:
    plt.savefig('/home/cat/correaltions_concatenated.svg',dpi=1200)
    plt.close()


IA1 Feb1_ [ -4.2         -6.63333333  -4.36666667  -4.63333333  -4.86666667
 -11.43333333 -10.13333333  -6.66666667  -4.43333333  -5.06666667
  -4.2         -4.16666667  -4.33333333  -4.4         -6.33333333
  -4.7         -3.86666667  -4.3         -3.56666667  -3.9
  -3.36666667  -3.96666667  -3.56666667  -3.73333333  -3.86666667
  -3.63333333  -4.26666667  -3.3         -2.86666667  -3.96666667
  -3.66666667  -4.13333333  -3.36666667  -3.26666667  -2.83333333
  -3.63333333  -3.03333333  -3.          -2.7         -3.
  -2.9         -2.66666667  -3.6         -2.43333333  -3.
  -2.46666667  -2.53333333  -2.5         -2.5         -2.16666667
  -2.          -2.          -2.        ]
all preds:  (53, 1)
IA1 Feb1_ [-4.53333333 -4.33333333 -3.13333333 -2.63333333 -2.66666667 -2.
 -2.8        -2.4        -3.13333333 -3.66666667 -2.         -2.
 -2.         -4.96666667 -4.8        -4.83333333 -4.76666667 -5.2
 -8.63333333 -7.36666667 -7.4        -7.2        -7.8        -6.9
 -6.         -4.7   

In [87]:
###################################################
############# PLOT PREDICTOIN TRENDS ##############
###################################################

# FIG 3I  3J
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

auc = []
fig, ax = plt.subplots()

for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    smooth = True
    smooth_window = 30
    auc.append([])
    for ctr,fname in enumerate(fnames):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']
        #print (acc.shape)

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)

            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                 linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())

# 
fig2=plt.figure()
for k in range(len(auc)):
    plt.plot(auc[k], label=names[k],
            linewidth=4)
plt.legend(fontsize=20)
plt.show()

(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)

In [88]:
#########################################
########### AUC AND EARLY PLOTS #########
#########################################

# FIg 3J 3I

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig2=plt.figure()
for k in range(len(early)):
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])
    
    #
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             label=ids[k]+ " ***** ",

             c=colors[k])

    # 
    corr = scipy.stats.pearsonr(t,temp2)
    print ("corr: ", corr)
    
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                linewidth=4,
                c=colors[k],
                alpha=.3)
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)


#
plt.xlim(0,80)
plt.ylim(-12,0)
plt.legend(fontsize=20)
plt.show()


corr:  (1.0, 0.0)
corr:  (-0.9999999999999999, 3.3310483861078736e-244)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-0.9999999999999999, 0.0)


In [169]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components.npy')
print (data.shape)

(40000, 30)


In [69]:
##############################################
##############################################
##############################################

from scipy.signal import savgol_filter

codes = ['Retrosplenial', 'barrel', 'motor', 'limb', 'visual']
clrs=['black','blue','red','magenta','brown']
smoothing = True
min_trials = 30
    
######################    
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'AQ2'

sessions = Visualize.get_sessions(root_dir,animal_id,'all')

print ("SESSIONS: ", sessions)
ctr_sess=0
fig = plt.figure()
for session_id in sessions:
    fnames = []
    for k in range(len(codes)):
        fnames.append(root_dir+ animal_id+'/SVM_Scores/SVM_Scores_ROI_'+
                      session_id+"_"+
                      codes[k]+'_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')

    try:
        temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,
                              session_id+"_all_locs_selected.txt"), dtype='str')
    except:
        print (" ntrial file missing: ")
        continue
        
    if len(temp)<min_trials:
        print ("too few trials: ", len(temp))
        continue

    ax=plt.subplot(8,10,ctr_sess+1)
    for ctr,fname in enumerate(fnames):
        try:
            data = np.load(fname, allow_pickle=True)
        except:
            print ("SVM not compued yet, skipping")
            continue

        acc_r = data['accuracy']

        if smoothing:
            for k in range(acc_r.shape[1]):
                acc_r[:,k] = savgol_filter(acc_r[:,k], 7, 3)

        #
        mean_r = acc_r[:900].mean(1)
        std_r = np.std(acc_r[:900],axis=1)

        t = np.arange(mean_r.shape[0])/30-29

        plt.plot(t,mean_r, linewidth=3,
                label=codes[ctr])
        if False:
            ax.fill_between(t, mean_r+std_r,
                           mean_r-std_r,
                           color=clrs[ctr],
                           alpha=.2)
    # 
    if ctr_sess==0:
        plt.legend()
    else:
        plt.xticks([])
        plt.yticks([])

    plt.ylabel("# trials: "+str(len(temp)))
    plt.xlim(-5,0)
    plt.ylim(0,1.0)
    plt.plot([-30,0],[0.5,0.5],'--',c='grey')

    ctr_sess+=1
   #except:
   #     pass
    
plt.show()



SESSIONS:  ['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Ja

<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec30_30Hz/AQ2am_Dec30_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


too few trials:  4
too few trials:  27
too few trials:  18
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
too few trials:  26
 ntrial file missing: 
too few trials:  13
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  4
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  21
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
too few trials:  0


<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb25_30Hz/AQ2am_Feb25_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping


In [33]:
data = np.load('/media/cat/4TBSSD/yuki/IJ1/SVM_Scores/SVM_Scores_IJ1pm_Feb10_30Hzcode_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow180.npz')
acc = data['accuracy']
print (acc.shape)

(1621, 10)
